In [2]:
import pandas as pd
import requests
import time

In [3]:
# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 

# 소환사명입니다.
summoner_name = 'hide on bush'

In [4]:
# SUMMONER-V4 API URL
requesturl = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summoner_name+\
'?api_key='+api_key

In [5]:
# puuid입니다.
summoner_puuid = requests.get(requesturl).json()['puuid']

In [6]:
# 솔랭 게임 목록 가져오기

games_list = []
index = 0
while True:
    requesturl = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+'/ids?queue=420&start='+str(index)+'&count=100&api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_list_new = response.json()
            break
    
    games_list.extend(game_list_new)
    
    if len(game_list_new) != 100:
        break
        
    index += 100
    time.sleep(1)

In [7]:
games_list

['EUW1_6564719806',
 'EUW1_6425882845',
 'EUW1_6418370296',
 'EUW1_6418295320',
 'EUW1_6418241766',
 'EUW1_6415307912',
 'EUW1_6415252341',
 'EUW1_6415188500',
 'EUW1_6415129288',
 'EUW1_6415002828',
 'EUW1_6414928754',
 'EUW1_6414857179',
 'EUW1_6413324261',
 'EUW1_6403711722',
 'EUW1_6403265073',
 'EUW1_6403197993',
 'EUW1_6397758519',
 'EUW1_6397714197',
 'EUW1_6395017722',
 'EUW1_6267567570',
 'EUW1_6140677307',
 'EUW1_6004056729',
 'EUW1_6004022705',
 'EUW1_5947551878',
 'EUW1_5884803531',
 'EUW1_5884551530',
 'EUW1_5862191902',
 'EUW1_5862044944',
 'EUW1_5860472221',
 'EUW1_5860327807',
 'EUW1_5860362598',
 'EUW1_5859457179',
 'EUW1_5859431054',
 'EUW1_5853341401',
 'EUW1_5853005211',
 'EUW1_5852918693',
 'EUW1_5851966071',
 'EUW1_5851962283',
 'EUW1_5851895387',
 'EUW1_5851316793',
 'EUW1_5851139721',
 'EUW1_5851152201',
 'EUW1_5846663723',
 'EUW1_5840498971',
 'EUW1_5840485804',
 'EUW1_5762155004',
 'EUW1_5762099850',
 'EUW1_5553788565',
 'EUW1_5553782252',
 'EUW1_5549156185',


In [8]:
# 솔랭 게임 각각에 대해서 API 호출 및 row binding
games_df = pd.DataFrame()
for matchId in games_list:
    requesturl = 'https://europe.api.riotgames.com/lol/match/v5/matches/'+matchId+'?api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_json = response.json()
            break
    game_df = pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]
    game_df = game_df[game_df.puuid == summoner_puuid][['championId','win','kills','deaths','assists']]
        
    games_df = pd.concat([games_df,game_df])
    time.sleep(1)

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]

,puuid,championId,win,kills,deaths,assists
0,HQoVrhD1DLA_iy9fvN0qct9fgRUXF4W1TVVttfMQnQAlZr...,14,False,6,6,4
1,PRtpct97pwWnhYplnUAMhYsZfMrxMRCBQenc_hY-KIGnb5...,56,False,1,7,4
2,jcKECH5UtmHSe35AQxdi0gFvc3eOTLvmC9FEzftK7YjtFk...,161,False,3,6,5
3,xbnQVGkYZLQwo3-b_yTkXohuRFTBE8S66CoSOWNIjwi1yR...,222,False,5,10,7
4,j8OV1SmIAlBtGf5FJ3hx6oXp8e_GvsOmZyQp62IlkHvJMx...,53,False,3,6,7
5,6NUiyiPGOMNVoU5whrz4t5-kVI0pPcRC7Pp8RvEHmnEkfH...,75,True,5,6,6
6,hJHHvuJA4ORws5QeYYaKBY09gggnoWImsQX7JHciulPEpu...,121,True,9,5,4
7,zevIjnRezLPFagCWgvzA1AX0i23Jy93QPTKBYU7GbROR3d...,90,True,7,1,12
8,BZWWjZpKLJKrYjoCel2bn_NMUoWSo31r3hrwpz24hc7LaW...,119,True,13,4,5
9,bXzKF71HsTtvtMTf7cW37C7G0U9oeU8jT1wJkoOZ6ft13R...,117,True,1,2,18


In [ ]:
games_df = games_df.groupby(['championId'],as_index=False).\
apply(lambda x: pd.Series({'win_count':sum(x.win),
                           'count':len(x),
                           'kills':sum(x.kills),
                           'deaths':sum(x.deaths),
                           'assists':sum(x.assists)
                          }))

In [ ]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [ ]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [ ]:
games_df = games_df.merge(champion_df,
                          how = 'left',
                          left_on = 'championId',
                          right_on = 'key'
                         )[['name','count','win_count','kills','deaths','assists']]

In [ ]:
games_df['WR'] = games_df['win_count'] / games_df['count']
games_df['KDA'] = (games_df['kills']+games_df['assists']) / games_df['deaths']

In [ ]:
games_df.sort_values('count',ascending = False)[['name','WR','count','KDA']]

,name,WR,count,KDA
10,드레이븐,0.619048,63,2.337079
12,야스오,0.520000,25,2.070796
5,카시오페아,0.560000,25,1.782051
8,그레이브즈,0.444444,9,3.000000
13,루시안,0.666667,9,2.500000
9,빅토르,0.500000,4,1.521739
11,신드라,0.750000,4,2.080000
7,리븐,0.666667,3,3.230769
16,요네,1.000000,2,4.428571
1,트위치,1.000000,1,2.714286
